In [1]:
# !rm -rf ../data/df_raw/*
# !gsutil -m cp -r gs://np-training-tmp/stackoverflow/posts_processed/*.parquet ../data/raw/

In [2]:
import modin.pandas as pd
import re
import lxml.html


In [3]:
df_raw = pd.read_parquet("../data/df_raw")


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



In [4]:
df_raw.head()

,Id,AcceptedAnswerId,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerBody
0,19419450,19419843,How to handle encrypted URL's in rails?,"<p>I am sending email to user, in that email o...",<ruby><ruby-on-rails-3><ruby-on-rails-3.2><rub...,1900,2,0,1,2013-10-17 06:11:50.320,<p>Life will be easier if you can do a slight ...
1,65367299,65367349,Creating html tags inside html tags using java...,<p>I'm trying to show data read from a JSON fi...,<javascript><html>,52,3,1,0,2020-12-19 06:33:32.643,<p>Do you want something like this?</p>\n<pre>...
2,32771510,32832061,Exercises in programming style with Scala,"<p>I've started to read the ""<a href=""https://...",<scala><coding-style>,328,2,1,1,2015-09-24 21:50:46.677,<p>Sure you can abstain from introducing varia...
3,2392236,2392314,javascript hide code for secret keys etc,<p>I am using the Ribbit javascript api and tr...,<javascript><api><secret-key>,1351,1,0,3,2010-03-06 11:03:00.030,<p>You can't. All JS is public viewable.</p>\n...
4,14220966,14221209,How to pass a template to a view in Backbone,<p>I'm trying to pass a template to my view. ...,<backbone.js><backbone-views>,4559,1,0,1,2013-01-08 17:43:02.843,<p>The way you are doing it in <code>App.Views...


In [5]:
len(df_raw)

11588225

In [6]:
def strip_html(s:str):
    try:
        return str(lxml.html.fromstring(s).text_content())
    except:
        return ''

def parse_tags(content:str):
    res =  re.findall(r'<(.+?)>',content)
    
    return ",".join(res)


In [7]:
strip_html(f"""

<p>I was asked to create a singleton that will...	
""")

'I was asked to create a singleton that will...\t\n'

In [8]:
df = df_raw

In [9]:
df['Body'] =  df['Body'].apply(strip_html)

In [10]:
df['AnswerBody'] =  df['AnswerBody'].apply(strip_html)

In [11]:
df['Tags'] = df['Tags'].apply(parse_tags)

In [12]:
!rm -rf ../data/df_processed/

In [13]:
df.to_parquet("../data/df_processed/")